In [1]:
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPO
from ray.tune.registry import register_env
from MA_env import MAEnvironment  # 确保正确导入您的环境类

# 创建环境函数
def env_creator(config):
    return MAEnvironment(num_agents=config.get("num_agents", 5))

# 注册环境
register_env("ma_env", env_creator)

# 定义共享策略的映射函数
def shared_policy_mapping_fn(agent_id, *args, **kwargs):
    return "shared_policy"

# 启动 Ray
ray.init()

# 配置
config = {
    "env": "ma_env",  # 使用注册的环境名
    "env_config": {
        "num_agents": 5,  # 传递环境的配置参数
    },
    "multiagent": {
        "policies": {
            "shared_policy": (None,  # 使用默认模型
                              env_creator({"num_agents": 5}).observation_space("agent_0"),  # 观测空间
                              env_creator({"num_agents": 5}).action_space("agent_0"),  # 动作空间
                              {}),
        },
        "policy_mapping_fn": shared_policy_mapping_fn,  # 使用共享策略映射
    },
    "framework": "torch",  # 使用 "torch" 或 "tf"
    "num_workers": 1,  # 使用的工作线程数
    "train_batch_size": 200,
    "sgd_minibatch_size": 64,
    "lr": 0.0003,
    "num_sgd_iter": 10,
}

# 使用 PPO 训练器进行训练
tune.run(PPO, config=config, stop={"training_iteration": 100})

# 关闭 Ray
ray.shutdown()

2024-11-17 20:47:49,583	INFO worker.py:1752 -- Started a local Ray instance.
2024-11-17 20:47:50,108	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-11-17 20:47:54,145	ERROR tune_controller.py:1332 -- Trial task failed for trial PPO_ma_env_27534_00000
Traceback (most recent call last):
  File "/Users/cyj/anaconda3/envs/py38/lib/python3.8/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/Users/cyj/anaconda3/envs/py38/lib/python3.8/site-packages/ray/_private/auto_init_hook.py", line 21, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/Users/cyj/anaconda3/envs/py38/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/Users/cyj/anaconda3/envs/py38/lib/python3.8/site-packages/ray/_private/worker.py", line 2667, in get
    values, debugger_breakpoint = worker.get_objects(object_refs, timeout=timeout)
  File "/Users/cyj/anaconda3/envs/py38/lib/python3.8/site-packages/ray/_private/worker.py", line 866, in get_objects
    raise value
ray.exceptions.RayActorError: The actor die

Trial name
PPO_ma_env_27534_00000


2024-11-17 20:47:54,149	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-11-17 20:47:54,150	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/Users/cyj/ray_results/PPO_2024-11-17_20-47-50' in 0.0022s.
(PPO pid=67968) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::PPO.__init__() (pid=67968, ip=127.0.0.1, actor_id=2d65674f6f97c5f9a612a8de01000000, repr=PPO)
(PPO pid=67968)   File 

TuneError: ('Trials did not complete', [PPO_ma_env_27534_00000])